In [1]:
import os
import pyspark
conf = pyspark.SparkConf()

# conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True) # enabled for debuggig 
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

/usr/local/spark/python/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# from pyspark.sql.functions import col, column, sum, dayofweek, udf, count, rank, collect_list, concat_ws, concat, lit, split, struct, trim, explode, lower, regexp_replace
# from pyspark.sql.types import IntegerType, DoubleType
# from pyspark.sql.window import Window
# from pyspark.ml.feature import NGram

In [8]:
# read
arrest_data_ytd_df = spark.read.option("header", "true").csv("data/NYPD_Arrest_Data__Year_to_Date__20231212.csv")
arrest_data_ytd_df.limit(5)

ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
261209118,01/01/2023,109,"ASSAULT 2,1,UNCLA...",106,FELONY ASSAULT,PL 1200501,F,K,77,0,45-64,F,BLACK,999335,186085,40.677426,-73.945615,POINT (-73.945615...
262984267,02/03/2023,515,CONTROLLED SUBSTA...,117,DANGEROUS DRUGS,PL 2203901,F,K,73,0,25-44,M,BLACK,1009318,178259,40.655923,-73.90965,POINT (-73.90965 ...
263664549,02/15/2023,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,K,62,0,25-44,M,WHITE,982272,158771,40.602468,-74.00712,POINT (-74.00712 ...
261345231,01/04/2023,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,M,32,0,25-44,M,BLACK,999899,238684,40.821797,-73.943457,POINT (-73.943457...
263536618,02/13/2023,109,"ASSAULT 2,1,UNCLA...",106,FELONY ASSAULT,PL 12005WX,F,K,71,0,25-44,M,BLACK,1001437,183080,40.669175,-73.938042,POINT (-73.938042...


In [7]:
arrest_data_historic_df = spark.read.option("header", "true").csv("data/NYPD_Arrests_Data__Historic__20231212.csv")
arrest_data_historic_df.limit(5)

ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
236791704,11/22/2021,581,NULL,NULL,NULL,PL 2225001,M,M,28,0,45-64,M,BLACK,997427,230378,40.799008797000056,-73.95240854099995,POINT (-73.952408...
237354740,12/04/2021,153,RAPE 3,104,RAPE,PL 1302502,F,B,41,0,25-44,M,WHITE HISPANIC,1013232,236725,40.816391847000034,-73.89529641399997,POINT (-73.895296...
236081433,11/09/2021,681,"CHILD, ENDANGERIN...",233,SEX CRIMES,PL 2601001,M,Q,113,0,25-44,M,BLACK,1046367,186986,40.67970040800003,-73.77604736799998,POINT (-73.776047...
32311380,06/18/2007,511,CONTROLLED SUBSTA...,235,DANGEROUS DRUGS,PL 2200300,M,Q,27,1,18-24,M,BLACK,NULL,NULL,NULL,NULL,NULL
192799737,01/26/2019,177,SEXUAL ABUSE,116,SEX CRIMES,PL 1306503,F,M,25,0,45-64,M,BLACK,1000555,230994,40.800694331000045,-73.941109285999971,POINT (-73.941109...
